In [1]:
### SCRIPT TO GET THE DATA FROM SNOWFLAKE AND DUMP THE FILE IN ONEDRIVE ###

In [2]:
### CUSTOMERS ###

In [3]:
#IMPORT LIBRARIES
import pandas as pd

In [4]:
#IMPORT LIBRARIES
import snowflake.connector

In [5]:
#CONNECT TO SNOWFLAKE WITH REPORTER CREDENTIALS
conn = snowflake.connector.connect(
    user='emirkabasoglu',
    password='7Bb13287',
    account='se63477.north-europe.azure',
    warehouse='REPORTING_WH',
    database='PROD_ANALYTICS_DB'
    )

In [6]:
#START THE CURSOR AND RUN THE QUERY
cur = conn.cursor()
cur.execute('''

WITH FTE_SIZE_SEGMENT AS (SELECT t2.customer_id,t3.fte_size,t7.segment
                            FROM prod_analytics_db.dbt_deploy.customers t2
                            LEFT JOIN dbt_deploy.stg_hubspot__company t3 on t3.company_id=t2.hubspot_company_id
                            JOIN common_db.static.segment AS t7 ON t3.fte_size BETWEEN t7.min_value AND t7.max_value
),

Platfrom_Segment as	( 	SELECT
						customer_id,
						CASE WHEN Platforms LIKE '%People%' AND Platforms NOT LIKE '%Infinite%' THEN 'People'
						 	 WHEN Platforms LIKE '%Infinite%' AND Platforms NOT LIKE '%People%' THEN 'Infinite'
						 	 WHEN Platforms LIKE '%People%' AND Platforms LIKE '%Infinite%' 	THEN 'People & Infinite'
						 	 ELSE 'No Platform' END AS Platfrom_segment
						FROM(
							SELECT
								customer_id,
								listagg(id_of_products,',') as Platforms	
					 	FROM(
							SELECT
							 T1.customer_id ,
							 T2.subscription_id,
							CASE WHEN T2.item_id IN ('1_1','1_2','1_3','1_4','1_5','1_6','1_7','8_1','8_2','8_3') THEN 'People'
								 WHEN T2.item_id IN ('2_1','2_2','2_3','2_4','2_11','8_4','8_5') THEN 'Infinite'
								 ELSE 'No Platform' END AS 	id_of_products
							 FROM prod_analytics_db.dbt_deploy.customers T1
								LEFT JOIN prod_analytics_db.dbt_deploy.subscriptions T2 ON T1.customer_id = T2.customer_id
								WHERE t2.status != 'cancelled' ) as sub_Table_1
							GROUP BY 1 ) AS Product_segment
										),
T_SHIRT_SIZE_SEGMENT AS (SELECT
                        CUSTOMER_ID,
                        CASE WHEN MAX_LICENSE BETWEEN 0 AND 99 	    and ACCOUNT_STATUS != 'cancelled' THEN 'XS'
		                     WHEN MAX_LICENSE BETWEEN 100 AND 499   and ACCOUNT_STATUS != 'cancelled' THEN 'S'
                    		 WHEN MAX_LICENSE BETWEEN 500 AND 999    and ACCOUNT_STATUS != 'cancelled' THEN 'M'
		                     WHEN MAX_LICENSE BETWEEN 1000 AND 4999 and ACCOUNT_STATUS != 'cancelled' THEN 'L'
		                     WHEN MAX_LICENSE BETWEEN 5000 AND 9999 and ACCOUNT_STATUS != 'cancelled' THEN 'XL'
		                     WHEN MAX_LICENSE >= 10000 			    and ACCOUNT_STATUS != 'cancelled' THEN 'XXL'
		                     WHEN ACCOUNT_STATUS != 'cancelled'	THEN 'No License'ELSE  NULL END AS license_Segment
                        
                        FROM(
                        SELECT
                        T1.CUSTOMER_ID,ACCOUNT_STATUS, MAX(T2.QUANTITY) AS MAX_LICENSE
                        FROM prod_analytics_db.dbt_deploy.CUSTOMERS T1
                        LEFT JOIN prod_analytics_db.dbt_deploy.SUBSCRIPTIONS T2 ON T1.CUSTOMER_ID =T2.CUSTOMER_ID  WHERE STATUS != 'cancelled' GROUP BY 1,2 ) AS SUB_TABLE )
select 
t1.*,
t2.fte_size,
t2.sub_industry, 
T3.Platfrom_segment,
T8.license_Segment,
t7.segment,
t4.ACCOUNT_MANAGER as am
FROM prod_analytics_db.dbt_deploy.customers t1
LEFT JOIN prod_analytics_db.dbt_deploy.stg_hubspot__company t2 on t1.hubspot_company_id =t2.company_id
LEFT JOIN prod_analytics_db.dbt_deploy.int_customer_csm_am_assigned t4 on t4.customer_id = t1.customer_id
LEFT JOIN Platfrom_Segment T3 ON T3.CUSTOMER_ID= T1.CUSTOMER_ID
LEFT JOIN FTE_SIZE_SEGMENT t7 on t7.customer_id =t1.customer_id
LEFT JOIN T_SHIRT_SIZE_SEGMENT t8 on t8.customer_id =t1.customer_id


            ''')


In [7]:
# ADD DESCRIPTION AS COLUMN LABELS
items_db_fetched = cur.fetchall()
cols = []
for i in cur.description:
    cols.append(i[0])

df_items_db_fetched = pd.DataFrame(items_db_fetched, columns=cols)

In [8]:
#MATCH DF OUTPUT TO DF MAIN
df_main = df_items_db_fetched

In [9]:
#DROP TIME RELATED COLUMNS
columns_to_drop = ['TZ_DK_UPDATED_AT', 'TZ_DK_CREATED_AT']
df_main = df_main.drop(columns_to_drop, axis=1)

In [10]:
#IMPORT LIBRARIES AND GET TODAY'S DATE 
from datetime import date
from datetime import datetime
from dateutil.relativedelta import relativedelta
#import pyodbc
from openpyxl import Workbook
report_date = date.today()

In [11]:
#CREATE THE OUTPUT FILE PATHNAME
output_folder = "/Users/emirkabasoglu/Library/CloudStorage/OneDrive-CeridianHCMInc/Data and Analytics/Revenue_Data_QA/Customers/"
output_filename = f"{report_date}-CUSTOMERS.xlsx"
output_filepath = output_folder + output_filename

In [12]:
#SAVE THE OUTPUT FILE TO THE WANTED FOLDER
with pd.ExcelWriter(output_filepath, engine='openpyxl') as writer:
    df_main.to_excel(writer, index=False, sheet_name='data-dump')

print("DataFrame exported to Excel successfully.")

DataFrame exported to Excel successfully.


In [13]:
### SUBSCRIPTIONS ###

In [14]:
#START THE CURSOR AND RUN THE QUERY
cur = conn.cursor()
cur.execute('''

SELECT 
t4.economic_id,t4.customer_name,T6.billing_period_frequency,t6.billing_period_unit,
t1.*,t2.EXTERNAL_NAME,T5.CURRENCY_CODE, T5.CALCULATED_ARR,
CASE WHEN T3.FAMILY_ID IN ('3') THEN 'content' 
     ELSE 'software' END AS product
FROM PROD_ANALYTICS_DB.DBT_DEPLOY.SUBSCRIPTIONS T1
LEFT JOIN PROD_ANALYTICS_DB.DBT_DEPLOY.PRODUCTS T2 ON T1.ITEM_ID =T2.ITEM_ID
LEFT JOIN PROD_ANALYTICS_DB.DBT_DEPLOY.PRODUCT_CATEGORIES T3 ON T2.ITEM_FAMILY_ID =T3.FAMILY_ID
LEFT JOIN PROD_ANALYTICS_DB.DBT_DEPLOY.CUSTOMERS T4 ON T4.CUSTOMER_ID = T1.CUSTOMER_ID
LEFT JOIN PROD_ANALYTICS_DB.DBT_DEPLOY.INT_ARR_CONVERTED_TO_EUR T5 ON T5.SUBSCRIPTION_ID  = T1.SUBSCRIPTION_ID
LEFT JOIN PROD_ANALYTICS_DB.DBT_DEPLOY.INT_FINAL_PRICES_NORMALIZED_ARR T6 ON T6.SUBSCRIPTION_ID=T1.SUBSCRIPTION_ID
    ''')


In [15]:
# ADD DESCRIPTION AS COLUMN LABELS
items_db_fetched = cur.fetchall()
cols = []
for i in cur.description:
    cols.append(i[0])

df_items_db_fetched = pd.DataFrame(items_db_fetched, columns=cols)

In [16]:
#MATCH DF OUTPUT TO DF MAIN
df_main = df_items_db_fetched

In [17]:
#IMPORT LIBRARIES AND GET TODAY'S DATE 
from datetime import date
from datetime import datetime
from dateutil.relativedelta import relativedelta
#import pyodbc
from openpyxl import Workbook
report_date = date.today()

In [18]:
#CREATE THE OUTPUT FILE PATHNAME
output_folder = "/Users/emirkabasoglu/Library/CloudStorage/OneDrive-CeridianHCMInc//Data and Analytics/Revenue_Data_QA/Subscriptions/"
output_filename = f"{report_date}-SUBSCRIPTIONS.xlsx"
output_filepath = output_folder + output_filename

In [19]:
#SAVE THE OUTPUT FILE TO THE WANTED FOLDER
with pd.ExcelWriter(output_filepath, engine='openpyxl') as writer:
    df_main.to_excel(writer, index=False, sheet_name='data-dump')

print("DataFrame exported to Excel successfully.")

DataFrame exported to Excel successfully.


In [20]:
#### STATIC TABLES ####

In [21]:
#PRODUCTS
cur = conn.cursor()
cur.execute('''
SELECT 
* 
FROM PROD_ANALYTICS_DB.DBT_DEPLOY.PRODUCTS
            ''')

# ADD DESCRIPTION AS COLUMN LABELS
items_db_fetched = cur.fetchall()
cols = []
for i in cur.description:
    cols.append(i[0])

df_items_db_fetched = pd.DataFrame(items_db_fetched, columns=cols)
#MATCH DF OUTPUT TO DF MAIN
df_main = df_items_db_fetched

#CREATE THE OUTPUT FILE PATHNAME
output_folder = "/Users/emirkabasoglu/Library/CloudStorage/OneDrive-CeridianHCMInc//Data and Analytics/Revenue_Data_QA/Products/"
output_filename = f"{report_date}-PRODUCTS.xlsx"
output_filepath = output_folder + output_filename

#SAVE THE OUTPUT FILE TO THE WANTED FOLDER
with pd.ExcelWriter(output_filepath, engine='openpyxl') as writer:
    df_main.to_excel(writer, index=False, sheet_name='data-dump')

print("DataFrame exported to Excel successfully.")

DataFrame exported to Excel successfully.


In [22]:
#PRODUCT_CATEGORIES
cur = conn.cursor()
cur.execute('''
SELECT 
* 
FROM PROD_ANALYTICS_DB.DBT_DEPLOY.PRODUCT_CATEGORIES
            ''')

# ADD DESCRIPTION AS COLUMN LABELS
items_db_fetched = cur.fetchall()
cols = []
for i in cur.description:
    cols.append(i[0])

df_items_db_fetched = pd.DataFrame(items_db_fetched, columns=cols)
#MATCH DF OUTPUT TO DF MAIN
df_main = df_items_db_fetched

#CREATE THE OUTPUT FILE PATHNAME
output_folder = "/Users/emirkabasoglu/Library/CloudStorage/OneDrive-CeridianHCMInc//Data and Analytics/Revenue_Data_QA/Products/"
output_filename = f"{report_date}-PRODUCT_CATEGORIES.xlsx"
output_filepath = output_folder + output_filename

#SAVE THE OUTPUT FILE TO THE WANTED FOLDER
with pd.ExcelWriter(output_filepath, engine='openpyxl') as writer:
    df_main.to_excel(writer, index=False, sheet_name='data-dump')

print("DataFrame exported to Excel successfully.")

DataFrame exported to Excel successfully.


In [23]:
#COUNTRY_CODE_REGION
cur = conn.cursor()
cur.execute('''
SELECT 
* 
FROM COMMON_DB.STATIC.COUNTRY_CODE_REGION
            ''')

# ADD DESCRIPTION AS COLUMN LABELS
items_db_fetched = cur.fetchall()
cols = []
for i in cur.description:
    cols.append(i[0])

df_items_db_fetched = pd.DataFrame(items_db_fetched, columns=cols)
#MATCH DF OUTPUT TO DF MAIN
df_main = df_items_db_fetched

#CREATE THE OUTPUT FILE PATHNAME
output_folder = "/Users/emirkabasoglu/Library/CloudStorage/OneDrive-CeridianHCMInc//Data and Analytics/Revenue_Data_QA/Static/"
output_filename = f"{report_date}-COUNTRY_CODE_REGION.xlsx"
output_filepath = output_folder + output_filename

#SAVE THE OUTPUT FILE TO THE WANTED FOLDER
with pd.ExcelWriter(output_filepath, engine='openpyxl') as writer:
    df_main.to_excel(writer, index=False, sheet_name='data-dump')

print("DataFrame exported to Excel successfully.")

DataFrame exported to Excel successfully.


In [24]:
#FX_RATE_FY24
cur = conn.cursor()
cur.execute('''
SELECT 
* 
FROM COMMON_DB.STATIC.FX_RATE_FY24
            ''')

# ADD DESCRIPTION AS COLUMN LABELS
items_db_fetched = cur.fetchall()
cols = []
for i in cur.description:
    cols.append(i[0])

df_items_db_fetched = pd.DataFrame(items_db_fetched, columns=cols)
#MATCH DF OUTPUT TO DF MAIN
df_main = df_items_db_fetched

#CREATE THE OUTPUT FILE PATHNAME
output_folder = "/Users/emirkabasoglu/Library/CloudStorage/OneDrive-CeridianHCMInc//Data and Analytics/Revenue_Data_QA/Static/"
output_filename = f"{report_date}-FX_RATE_FY24.xlsx"
output_filepath = output_folder + output_filename

#SAVE THE OUTPUT FILE TO THE WANTED FOLDER
with pd.ExcelWriter(output_filepath, engine='openpyxl') as writer:
    df_main.to_excel(writer, index=False, sheet_name='data-dump')

print("DataFrame exported to Excel successfully.")

DataFrame exported to Excel successfully.


In [25]:
#GICS
cur = conn.cursor()
cur.execute('''
SELECT 
* 
FROM COMMON_DB.STATIC.GICS
            ''')

# ADD DESCRIPTION AS COLUMN LABELS
items_db_fetched = cur.fetchall()
cols = []
for i in cur.description:
    cols.append(i[0])

df_items_db_fetched = pd.DataFrame(items_db_fetched, columns=cols)
#MATCH DF OUTPUT TO DF MAIN
df_main = df_items_db_fetched

#CREATE THE OUTPUT FILE PATHNAME
output_folder = "/Users/emirkabasoglu/Library/CloudStorage/OneDrive-CeridianHCMInc//Data and Analytics/Revenue_Data_QA/Static/"
output_filename = f"{report_date}-GICS.xlsx"
output_filepath = output_folder + output_filename

#SAVE THE OUTPUT FILE TO THE WANTED FOLDER
with pd.ExcelWriter(output_filepath, engine='openpyxl') as writer:
    df_main.to_excel(writer, index=False, sheet_name='data-dump')

print("DataFrame exported to Excel successfully.")

DataFrame exported to Excel successfully.


In [26]:
#LEAD_SOURCE_GROUP
cur = conn.cursor()
cur.execute('''
SELECT 
* 
FROM COMMON_DB.STATIC.LEAD_SOURCE_GROUP
            ''')

# ADD DESCRIPTION AS COLUMN LABELS
items_db_fetched = cur.fetchall()
cols = []
for i in cur.description:
    cols.append(i[0])

df_items_db_fetched = pd.DataFrame(items_db_fetched, columns=cols)
#MATCH DF OUTPUT TO DF MAIN
df_main = df_items_db_fetched

#CREATE THE OUTPUT FILE PATHNAME
output_folder = "/Users/emirkabasoglu/Library/CloudStorage/OneDrive-CeridianHCMInc//Data and Analytics/Revenue_Data_QA/Static/"
output_filename = f"{report_date}-LEAD_SOURCE_GROUP.xlsx"
output_filepath = output_folder + output_filename

#SAVE THE OUTPUT FILE TO THE WANTED FOLDER
with pd.ExcelWriter(output_filepath, engine='openpyxl') as writer:
    df_main.to_excel(writer, index=False, sheet_name='data-dump')

print("DataFrame exported to Excel successfully.")

DataFrame exported to Excel successfully.


In [27]:
#SEGMENT
cur = conn.cursor()
cur.execute('''
SELECT 
* 
FROM COMMON_DB.STATIC.SEGMENT
            ''')

# ADD DESCRIPTION AS COLUMN LABELS
items_db_fetched = cur.fetchall()
cols = []
for i in cur.description:
    cols.append(i[0])

df_items_db_fetched = pd.DataFrame(items_db_fetched, columns=cols)
#MATCH DF OUTPUT TO DF MAIN
df_main = df_items_db_fetched

#CREATE THE OUTPUT FILE PATHNAME
output_folder = "/Users/emirkabasoglu/Library/CloudStorage/OneDrive-CeridianHCMInc//Data and Analytics/Revenue_Data_QA/Static/"
output_filename = f"{report_date}-SEGMENT.xlsx"
output_filepath = output_folder + output_filename

#SAVE THE OUTPUT FILE TO THE WANTED FOLDER
with pd.ExcelWriter(output_filepath, engine='openpyxl') as writer:
    df_main.to_excel(writer, index=False, sheet_name='data-dump')

print("DataFrame exported to Excel successfully.")

DataFrame exported to Excel successfully.


In [28]:
#SIZE
cur = conn.cursor()
cur.execute('''
SELECT 
* 
FROM COMMON_DB.STATIC.SIZE
            ''')

# ADD DESCRIPTION AS COLUMN LABELS
items_db_fetched = cur.fetchall()
cols = []
for i in cur.description:
    cols.append(i[0])

df_items_db_fetched = pd.DataFrame(items_db_fetched, columns=cols)
#MATCH DF OUTPUT TO DF MAIN
df_main = df_items_db_fetched

#CREATE THE OUTPUT FILE PATHNAME
output_folder = "/Users/emirkabasoglu/Library/CloudStorage/OneDrive-CeridianHCMInc//Data and Analytics/Revenue_Data_QA/Static/"
output_filename = f"{report_date}-SIZE.xlsx"
output_filepath = output_folder + output_filename

#SAVE THE OUTPUT FILE TO THE WANTED FOLDER
with pd.ExcelWriter(output_filepath, engine='openpyxl') as writer:
    df_main.to_excel(writer, index=False, sheet_name='data-dump')

print("DataFrame exported to Excel successfully.")

DataFrame exported to Excel successfully.


In [29]:
#TARGETS_FY24
cur = conn.cursor()
cur.execute('''
SELECT 
* 
FROM COMMON_DB.STATIC.TARGETS_FY24
            ''')

# ADD DESCRIPTION AS COLUMN LABELS
items_db_fetched = cur.fetchall()
cols = []
for i in cur.description:
    cols.append(i[0])

df_items_db_fetched = pd.DataFrame(items_db_fetched, columns=cols)
#MATCH DF OUTPUT TO DF MAIN
df_main = df_items_db_fetched

#CREATE THE OUTPUT FILE PATHNAME
output_folder = "/Users/emirkabasoglu/Library/CloudStorage/OneDrive-CeridianHCMInc//Data and Analytics/Revenue_Data_QA/Static/"
output_filename = f"{report_date}-TARGETS_FY24.xlsx"
output_filepath = output_folder + output_filename

#SAVE THE OUTPUT FILE TO THE WANTED FOLDER
with pd.ExcelWriter(output_filepath, engine='openpyxl') as writer:
    df_main.to_excel(writer, index=False, sheet_name='data-dump')

print("DataFrame exported to Excel successfully.")

DataFrame exported to Excel successfully.
